In [34]:
#!pip install stan
#!pip install nest_asyncio

In [35]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
import pystan
from tqdm.auto import tqdm
import stan # pystan3
import nest_asyncio # штука, чтобы pystan3 работал

nest_asyncio.apply()
warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 8
%precision 4
plt.style.use('_classic_test_patch')

In [36]:
# функция для генерации выборки
def generate_sample(size=100):
    x_0 = np.ones(shape=size)
    x_1 = np.random.normal(loc=3, scale=2, size=size)
    x_2 = np.random.uniform(low=1, high=3, size=size)
    X = np.vstack((x_0, x_1, x_2)).T
    
    betas = np.array([2, -1, 0.5])
    eps = np.random.normal(loc=0, scale=1, size=size) # предпосылка пробита

    y = (X @ betas + eps > 0) * 1

    probit_data = {'n': size,
                   'y': y,
                   'x_1': x_1,
                   'x_2': x_2
                  }
    
    return probit_data

print('Доля единиц в целевой переменной:', np.mean(generate_sample()['y']))

Доля единиц в целевой переменной: 0.52


In [37]:
code = '''
data {
    int<lower=0> n;
    int<lower=0,upper=1> y[n];
    vector[n] x_1;
    vector[n] x_2;
}

parameters {
    real beta0;
    real beta1;
    real beta2;
}

model {
    beta0 ~ normal(0, 10);
    beta1 ~ normal(0, 10);
    beta2 ~ normal(0, 10);
    y ~ bernoulli(Phi(beta0 + beta1 * x_1 + beta2 * x_2));
}

'''

In [38]:
from sklearn.metrics import mean_absolute_percentage_error as mape, mean_squared_error as mse

params = ['beta0', 'beta1', 'beta2']
betas = np.array([2, -1, 0.5])

def summary(code0, n_iter=100, est_func=np.mean, n=10):
    # сюда будем складывать оценки на симуляциях (по дефолту будет считаться как среднее)
    results = {'beta0': [],
               'beta1': [],
               'beta2': []
              }
    # сюда будем складывать метрики и размер выборки
    total = [n]
    # строим модели
    for epoch in tqdm(range(n_iter)):
        # генерируем данные
        data = generate_sample(size=n)
        # обучаемся
        posterior = stan.build(code0, data=data, random_seed=999)
        fit = posterior.sample(num_chains=1, num_samples=2000)
        # записываем результаты
        for i in range(len(betas)):
            estimator = est_func(fit[params[i]]) 
            results[params[i]].append(estimator)
    # подсчитываем метрики
    for i in range(len(params)):
        true_coef = np.ones(shape=n_iter) * betas[i]
        total.append([params[i], 
                      mape(true_coef, results[params[i]]) * 100, # в процентах
                      mse(true_coef, results[params[i]], squared=False)])
    return total

In [47]:
np.random.seed(999) # тут обучается 10 минут, но сохраняется в память
results_norm_prior = summary(code, n=1000)
results_norm_prior

  0%|          | 0/100 [00:00<?, ?it/s]

Building...

INFO:aiohttp.access:127.0.0.1 [15/Feb/2022:22:09:02 +0000] "POST /v1/models/r5vt2h6t/params HTTP/1.1" 200 386 "-" "Python/3.8 aiohttp/3.8.1"
INFO:httpstan:Found Stan model in cache (`models/r5vt2h6t`).
INFO:aiohttp.access:127.0.0.1 [15/Feb/2022:22:09:02 +0000] "POST /v1/models HTTP/1.1" 201 234 "-" "Python/3.8 aiohttp/3.8.1"



Building: found in cache, done.INFO:aiohttp.access:127.0.0.1 [15/Feb/2022:22:09:02 +0000] "POST /v1/models/r5vt2h6t/params HTTP/1.1" 200 386 "-" "Python/3.8 aiohttp/3.8.1"
Sampling:   0%INFO:aiohttp.access:127.0.0.1 [15/Feb/2022:22:09:02 +0000] "POST /v1/models/r5vt2h6t/fits HTTP/1.1" 201 325 "-" "Python/3.8 aiohttp/3.8.1"
INFO:aiohttp.access:127.0.0.1 [15/Feb/2022:22:09:02 +0000] "GET /v1/models/r5vt2h6t/fits/dke2mzjq HTTP/1.1" 200 1569347 "-" "Python/3.8 aiohttp/3.8.1"

Sampling: 100%, done.
Messages received during sampling:
  Rejecting initial value:
    Log probability evaluates to log(0), i.e. negative infinity.
    Stan can't start sampling from this initial value.
  Gradient evaluation took 0.000364 seconds
  1000 transitions using 10 leapfrog steps per transition would take 3.64 seconds.
  Adjust your expectations accordingly!


Building...

...


Building: found in cache, done.INFO:aiohttp.access:127.0.0.1 [15/Feb/2022:22:09:17 +0000] "POST /v1/models/r5vt2h6t/params HTTP/1.1" 200 386 "-" "Python/3.8 aiohttp/3.8.1"
Sampling:   0%INFO:aiohttp.access:127.0.0.1 [15/Feb/2022:22:09:17 +0000] "POST /v1/models/r5vt2h6t/fits HTTP/1.1" 201 325 "-" "Python/3.8 aiohttp/3.8.1"
INFO:aiohttp.access:127.0.0.1 [15/Feb/2022:22:09:17 +0000] "GET /v1/models/r5vt2h6t/fits/uqcaveyp HTTP/1.1" 200 1568844 "-" "Python/3.8 aiohttp/3.8.1"

Sampling: 100%, done.
Messages received during sampling:
  Rejecting initial value:
    Log probability evaluates to log(0), i.e. negative infinity.
    Stan can't start sampling from this initial value.
  Gradient evaluation took 0.000111 seconds
  1000 transitions using 10 leapfrog steps per transition would take 1.11 seconds.
  Adjust your expectations accordingly!


Building...

INFO:aiohttp.access:127.0.0.1 [15/Feb/2022:22:09:18 +0000] "POST /v1/models/r5vt2h6t/params HTTP/1.1" 200 386 "-" "Python/3.8 aiohttp/3.8.1"
INFO:httpstan:Found Stan model in cache (`models/r5vt2h6t`).
INFO:aiohttp.access:127.0.0.1 [15/Feb/2022:22:09:18 +0000] "POST /v1/models HTTP/1.1" 201 234 "-" "Python/3.8 aiohttp/3.8.1"



Building: found in cache, done.INFO:aiohttp.access:127.0.0.1 [15/Feb/2022:22:09:18 +0000] "POST /v1/models/r5vt2h6t/params HTTP/1.1" 200 386 "-" "Python/3.8 aiohttp/3.8.1"
Sampling:   0%INFO:aiohttp.access:127.0.0.1 [15/Feb/2022:22:09:18 +0000] "POST /v1/models/r5vt2h6t/fits HTTP/1.1" 201 325 "-" "Python/3.8 aiohttp/3.8.1"
INFO:aiohttp.access:127.0.0.1 [15/Feb/2022:22:09:18 +0000] "GET /v1/models/r5vt2h6t/fits/c5mavaee HTTP/1.1" 200 1568173 "-" "Python/3.8 aiohttp/3.8.1"

Sampling: 100%, done.
Messages received during sampling:
  Rejecting initial value:
    Log probability evaluates to log(0), i.e. negative infinity.
    Stan can't start sampling from this initial value.
  Gradient evaluation took 0.000109 seconds
  1000 transitions using 10 leapfrog steps per transition would take 1.09 seconds.
  Adjust your expectations accordingly!


[1000,
 ['beta0', 9.959104162513349, 0.2616013776019921],
 ['beta1', 5.185305499178744, 0.06563468976661484],
 ['beta2', 16.006018615651602, 0.09872159531630341]]

In [48]:
results_norm_prior

[1000,
 ['beta0', 9.959104162513349, 0.2616013776019921],
 ['beta1', 5.185305499178744, 0.06563468976661484],
 ['beta2', 16.006018615651602, 0.09872159531630341]]